In [8]:
#!/usr/bin/env python3
import numpy as np
from ikpy.chain import Chain
from ikpy.link import OriginLink, URDFLink
from dynamixel_sdk import PortHandler, PacketHandler

# ----------------------------
# Define the Kinematic Chain (in meters)
# ----------------------------
# Joint 7: Base rotation (rotates about z)
joint7 = URDFLink(
    name="7_base_rotation",
    origin_translation=np.array([0, 0, 0.038]),  # 38 mm upward from base
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([0, 0, 1]),
    joint_type="revolute",
    bounds=(0, 3*np.pi/2)
)

# Joint 8: Shoulder tilt (rotates about y, here using [0, -1, 0])
joint8 = URDFLink(
    name="8_shoulder_tilt",
    origin_translation=np.array([0, 0, 0.017]),  # 17 mm upward from Joint 7
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([0, -1, 0]),
    joint_type="revolute",
    bounds=(-.65, np.pi/2)
)

# Joint 9: Elbow tilt (rotates about y)
L2 = 0.110  # 110 mm from Joint 8
joint9 = URDFLink(
    name="9_elbow_tilt",
    origin_translation=np.array([0, 0, L2]),
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([0, -1, 0]),
    joint_type="revolute",
    bounds=(-110 / 180 * np.pi, 1.4)
)

# Joint 10: Wrist tilt (rotates about y)
L3 = 0.100  # 100 mm from Joint 9
joint10 = URDFLink(
    name="10_wrist_tilt",
    origin_translation=np.array([-L3, 0, 0]),
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([0, 1, 0]),
    joint_type="revolute",
    bounds=(-np.pi/2, np.pi/2)
)

# Joint 11: Gripper rotate (rotates about z)
L4 = 0.120 # 45 mm from Joint 10
joint11 = URDFLink(
    name="11_gripper_rotate",
    origin_translation=np.array([-L4, 0, 0]),
    origin_orientation=np.array([0, 0, 0]),
    rotation=np.array([1, 0, 0]),
    joint_type="revolute"
)

# Assemble the robot chain: The first link is the fixed base (OriginLink)
robot_chain_11 = Chain(name="my_robot_arm_upto_joint11", links=[
    OriginLink(),
    joint7,
    joint8,
    joint9,
    joint10,
    joint11
])

# ----------------------------
# Dynamixel Configuration
# ----------------------------
PROTOCOL_VERSION = 2.0
DXL_IDS = [7, 8, 9, 10, 11, 12]  # Use first 5 IDs for joints 7-11.
BAUDRATE = 1e6
DEVICENAME = '/dev/tty.usbmodem58FD0170871'  # Replace with your serial port

TICKS_PER_REV = 4096

def rad_to_dxl(angle_rad):
    angle_rad = angle_rad
    return int((angle_rad / (2 * np.pi)) * TICKS_PER_REV)

def dxl_to_rad(ticks):
    return (ticks / TICKS_PER_REV) * 2 * np.pi

# ----------------------------
# Initialize Port and Packet Handler (No torque enable required here)
# ----------------------------
port_handler = PortHandler(DEVICENAME)
packet_handler = PacketHandler(PROTOCOL_VERSION)
if not port_handler.openPort():
    print("Failed to open port!")
    exit()
if not port_handler.setBaudRate(BAUDRATE):
    print("Failed to set baudrate!")
    exit()

def to_signed(val):
    if val > 0x7FFFFFFF:  # 0x7FFFFFFF is 2^31 - 1
        return val - 0x100000000  # 0x100000000 is 2^32
    return val

# ----------------------------
# Read Current Angles Function
# ----------------------------
def read_current_angles():
    angles = []
    for dxl_id in DXL_IDS[:5]:  # Only for joints 7-11
        pos, dxl_comm_result, dxl_error = packet_handler.read4ByteTxRx(port_handler, dxl_id, 132)
        print("ID", dxl_id, "TICKS", pos)
        if dxl_comm_result != 0 or dxl_error != 0:
            print(f"Error reading servo {dxl_id}, assuming 0.")
            angles.append(0)
        else:
            angle = dxl_to_rad(to_signed(pos))
            print("RAW POS", pos)
            print("SIGNED POS", to_signed(pos))
            angles.append(angle)
    return np.array(angles)

# ----------------------------
# Forward Kinematics Function for Chain Up to Joint 11
# ----------------------------
def forward_kinematics_11(angles):
    truncated = angles[:5]  # angles for joints 7-11
    full_angles = np.concatenate(([0], truncated))  # Prepend dummy for OriginLink
    return robot_chain_11.forward_kinematics(full_angles)

# ----------------------------
# Main Execution: Read current angles, compute FK, and print position (in cm)
# ----------------------------
current_angles = read_current_angles()
print("Current joint angles (deg) from servos:", current_angles*180/ (np.pi))
print("Current joint angles (rad) from servos:", current_angles)

home_pose = forward_kinematics_11(current_angles)
home_xyz = home_pose[:3, 3] * 100  # Convert from meters to centimeters

print("\nHome position (xyz) of end-effector (joint 11) relative to base (in cm):")
print(f"  x = {home_xyz[0]:.2f} cm")
print(f"  y = {home_xyz[1]:.2f} cm")
print(f"  z = {home_xyz[2]:.2f} cm")

port_handler.closePort()


ID 7 TICKS 2559
RAW POS 2559
SIGNED POS 2559
ID 8 TICKS 578
RAW POS 578
SIGNED POS 578
ID 9 TICKS 4294966331
RAW POS 4294966331
SIGNED POS -965
ID 10 TICKS 4294967258
RAW POS 4294967258
SIGNED POS -38
ID 11 TICKS 1166
RAW POS 1166
SIGNED POS 1166
Current joint angles (deg) from servos: [224.91210938  50.80078125 -84.81445312  -3.33984375 102.48046875]
Current joint angles (rad) from servos: [ 3.92545684  0.8866409  -1.48029146 -0.05829127  1.7886216 ]

Home position (xyz) of end-effector (joint 11) relative to base (in cm):
  x = 19.22 cm
  y = 19.16 cm
  z = 24.17 cm


In [10]:
#======MOVE
import numpy as np
import time
from ikpy.chain import Chain
from ikpy.link import OriginLink, URDFLink
from dynamixel_sdk import PortHandler, PacketHandler

# ----------------------------
# Initialize Port and Packet Handler, and enable torque
# ----------------------------
port_handler = PortHandler(DEVICENAME)
packet_handler = PacketHandler(PROTOCOL_VERSION)

def to_unsigned(val):
    if val < 0:
        return val + 0x100000000  # 0x100000000 is 2^32
    return val


if not port_handler.openPort():
    print("Failed to open port!")
    exit()
if not port_handler.setBaudRate(BAUDRATE):
    print("Failed to set baudrate!")
    exit()

# Enable torque on servos for joints 7-11 (first 5 IDs)
TORQUE_ENABLE_ADDR = 64  # Typical address for torque enable in protocol 2.0
TORQUE_ENABLE_VALUE = 1
for dxl_id in DXL_IDS[:5]:
    comm_result, error = packet_handler.write1ByteTxRx(port_handler, dxl_id, TORQUE_ENABLE_ADDR, TORQUE_ENABLE_VALUE)
    if comm_result != 0 or error != 0:
        print(f"Error enabling torque for servo {dxl_id}: {comm_result}, {error}")
    else:
        print(f"Torque enabled for servo {dxl_id}")

# ----------------------------
# Function to Read Current Angles (radians) from Servos (joints 7-11)
# ----------------------------
def read_current_angles():
    angles = []
    for dxl_id in DXL_IDS[:5]:
        pos, comm_result, error = packet_handler.read4ByteTxRx(port_handler, dxl_id, 132)
        if comm_result != 0 or error != 0:
            print(f"Error reading servo {dxl_id}, assuming 0.")
            angles.append(0)
        else:
            angles.append(dxl_to_rad(to_signed(pos)))
    return np.array(angles)

# ----------------------------
# Forward Kinematics Function for Chain Up to Joint 11
# ----------------------------
def forward_kinematics_11(angles):
    """
    Computes FK for joints 7-11.
    Prepend a dummy value (0) for the base (OriginLink) and compute the 4x4 pose.
    """
    full_angles = np.concatenate(([0], angles))
    return robot_chain_11.forward_kinematics(full_angles)

# ----------------------------
# Build a Target Pose (in meters)
# ----------------------------
def build_target_pose(x, y, z):
    pose = np.eye(4)
    pose[:3, 3] = np.array([x, y, z])
    return pose

# ----------------------------
# Move to a Target Based on Current Position (using current configuration as initial guess)
# ----------------------------
def move_to_target_from_current(target_x_cm, target_y_cm, target_z_cm):
    # Read current servo angles (radians) for joints 7-11.
    current_angles = read_current_angles()
    print("Current joint angles (radians):", current_angles * 180/np.pi)
    
    # Full configuration: prepend 0 for the base.
    current_full_angles = np.concatenate(([0], current_angles))
    
    # Build target pose from user input (convert cm to m).
    target_pose = build_target_pose(target_x_cm / 100.0, target_y_cm / 100.0, target_z_cm / 100.0)
    print("Target pose matrix (meters):")
    print(target_pose)
    
    # Compute IK using current configuration as the initial guess.
    ik_solution = robot_chain_11.inverse_kinematics_frame(target_pose, initial_position=current_full_angles)
    # Extract joint angles for joints 7-11 (ignore the dummy base element at index 0).
    desired_angles = ik_solution[1:6]
    
    print("\nCalculated joint angles (radians) for target position:")
    # Print the angles directly in radians.
    for i, angle in enumerate(desired_angles):
        print(f"  Joint {7+i}: {angle*180/np.pi} degree")
    
    # Command servos (for joints 7-11) using the computed angles.
    for i, dxl_id in enumerate(DXL_IDS[:5]):
        target_tick = to_unsigned(rad_to_dxl(desired_angles[i]))
        print("WRITING TICKS SIGNED: ", rad_to_dxl(desired_angles[i]))
        print("TICKS UNSIGNED", target_tick)
        
        comm_result, error = packet_handler.write4ByteTxRx(port_handler, dxl_id, 116, target_tick)
        if comm_result != 0 or error != 0:
            print(f"Error sending command to servo {dxl_id}: {comm_result}, {error}")
        else:
            print(f"Servo {dxl_id} commanded to {desired_angles[i]:.3f} rad ({target_tick} ticks)")
    
    time.sleep(1)
    
    # Read back new servo angles and compute the new end-effector pose.
    new_angles = read_current_angles()
    new_pose = forward_kinematics_11(new_angles)
    new_xyz = new_pose[:3, 3] * 100  # convert from m to cm
    print("\nNew end-effector position (cm):")
    print(f"  x = {new_xyz[0]:.2f} cm, y = {new_xyz[1]:.2f} cm, z = {new_xyz[2]:.2f} cm")

# ----------------------------
# Main Interactive Loop
# ----------------------------
def main_loop():
    while True:
        current_angles = read_current_angles()
        current_pose = forward_kinematics_11(current_angles)
        current_xyz = current_pose[:3, 3] * 100  # in cm
        print("\nCurrent end-effector position (cm):")
        print(f"  x = {current_xyz[0]:.2f} cm, y = {current_xyz[1]:.2f} cm, z = {current_xyz[2]:.2f} cm")
        
        user_input = input("\nEnter new target coordinates (x y z in cm), or 'q' to quit: ")
        if user_input.strip().lower() == 'q':
            break
        try:
            tx, ty, tz = map(float, user_input.strip().split())
        except Exception:
            print("Invalid input. Please enter three numbers separated by spaces.")
            continue
        
        move_to_target_from_current(tx, ty, tz)
    
    port_handler.closePort()
    print("Port closed. Exiting.")

if __name__ == '__main__':
    main_loop()


Torque enabled for servo 7
Torque enabled for servo 8
Torque enabled for servo 9
Torque enabled for servo 10
Torque enabled for servo 11

Current end-effector position (cm):
  x = 20.59 cm, y = 20.53 cm, z = 20.95 cm
Current joint angles (radians): [224.91210938  56.68945313 -83.40820312  -2.54882812 102.48046875]
Target pose matrix (meters):
[[1.  0.  0.  0.1]
 [0.  1.  0.  0.1]
 [0.  0.  1.  0.3]
 [0.  0.  0.  1. ]]

Calculated joint angles (radians) for target position:
  Joint 7: 224.99999983154018 degree
  Joint 8: 47.64756012479087 degree
  Joint 9: -79.9266025169386 degree
  Joint 10: 69.46071407605041 degree
  Joint 11: 102.48046875000001 degree
WRITING TICKS SIGNED:  2559
TICKS UNSIGNED 2559
Servo 7 commanded to 3.927 rad (2559 ticks)
WRITING TICKS SIGNED:  542
TICKS UNSIGNED 542
Servo 8 commanded to 0.832 rad (542 ticks)
WRITING TICKS SIGNED:  -909
TICKS UNSIGNED 4294966387
Servo 9 commanded to -1.395 rad (4294966387 ticks)
WRITING TICKS SIGNED:  790
TICKS UNSIGNED 790
Servo 

In [220]:
#!/usr/bin/env python3
import numpy as np
from dynamixel_sdk import PortHandler, PacketHandler

# ----------------------------
# Configuration Parameters
# ----------------------------
DEVICENAME = '/dev/tty.usbmodem58FD0170871'  # Update with your serial port
BAUDRATE = 1000000  # 1e6
PROTOCOL_VERSION = 2.0

# List of Dynamixel servo IDs (adjust order to match your robot's joints)
DXL_IDS = [7, 8, 9, 10, 11, 12]

# Dynamixel resolution and neutral (home) position
TICKS_PER_REV = 4096
NEUTRAL_TICKS = 2048  # Adjust according to your servo's homing offset

# ----------------------------
# Conversion Functions
# ----------------------------
def rad_to_dxl(angle_rad):
    """
    Convert an angle (in radians) to the corresponding Dynamixel tick value.
    This function assumes that 0 rad corresponds to NEUTRAL_TICKS and
    2π rad corresponds to TICKS_PER_REV ticks above NEUTRAL_TICKS.
    """
    # Normalize the angle to [0, 2π)
    angle_rad = angle_rad % (2 * np.pi)
    ticks_offset = int((angle_rad / (2 * np.pi)) * TICKS_PER_REV)
    return NEUTRAL_TICKS + ticks_offset

def dxl_to_rad(ticks):
    """
    Convert a Dynamixel tick value to an angle in radians.
    """
    return ((ticks - NEUTRAL_TICKS) / TICKS_PER_REV) * 2 * np.pi

# ----------------------------
# Main Function: Connect and Command Servos
# ----------------------------
def main():
    # Initialize port and packet handler
    port_handler = PortHandler(DEVICENAME)
    packet_handler = PacketHandler(PROTOCOL_VERSION)
    
    if not port_handler.openPort():
        print("Failed to open port!")
        return
    if not port_handler.setBaudRate(BAUDRATE):
        print("Failed to set baudrate!")
        return

    # For demonstration, define a target angle (in radians) for each servo.
    # Here, we command every servo to 45° (approximately 0.785 rad).
    desired_angles = [0.785 for _ in DXL_IDS]

    # Loop over each servo, convert the angle to ticks, and send the command.
    for idx, angle in enumerate(desired_angles):
        dxl_id = DXL_IDS[idx]
        goal_pos = rad_to_dxl(angle)
        # Register 116 is commonly used for goal position; adjust if needed.
        dxl_comm_result, dxl_error = packet_handler.write4ByteTxRx(
            port_handler, dxl_id, 116, goal_pos
        )
        if dxl_comm_result != 0:
            print(f"Communication error on servo {dxl_id}: {packet_handler.getTxRxResult(dxl_comm_result)}")
        elif dxl_error != 0:
            print(f"Servo {dxl_id} error: {packet_handler.getRxPacketError(dxl_error)}")
        else:
            print(f"Servo {dxl_id} set to goal position {goal_pos}")

    # Optionally, add a delay to allow movement before closing the port.
    # time.sleep(2)

    # Close the port after sending commands.
    port_handler.closePort()

if __name__ == '__main__':
    main()


Servo 7 set to goal position 2559
Servo 8 set to goal position 2559
Servo 9 set to goal position 2559
Servo 10 set to goal position 2559
Servo 11 set to goal position 2559
Servo 12 set to goal position 2559
